<a href="https://colab.research.google.com/github/kuds/rl-lunar-lander/blob/main/%5BLunar%20Lander%5D%20Soft%20Actor-Critic%20(SAC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Soft Actor-Critic (SAC)
---
In this notebook, you will implement a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [ ]:
!pip install swig

In [ ]:
!pip install stable_baselines3 gymnasium gymnasium[box2d] rl_zoo3

In [ ]:
import gymnasium
import stable_baselines3
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage
from stable_baselines3.common.vec_env import VecVideoRecorder

import rl_zoo3
import torch
import numpy
import scipy

In [ ]:
print("Torch Version: {}".format(torch.__version__))
print("Numpy Version: {}".format(numpy.__version__))
print("Scipy Version: {}".format(scipy.__version__))
print("gymnasium Version: {}".format(gymnasium.__version__))
print("Stable Baselines3 Version: {}".format(stable_baselines3.__version__))

Torch Version: 2.3.1+cu121
Numpy Version: 1.26.4
Scipy Version: 1.13.1
gymnasium Version: 0.29.1
Stable Baselines3 Version: 2.3.2


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#env = gymnasium.make("LunarLander-v2", continuous=True)
env = make_vec_env("LunarLanderContinuous-v2", n_envs=16)

#env_val = gymnasium.make("LunarLander-v2", render_mode="rgb_array", continuous=True)
env_val = make_vec_env("LunarLanderContinuous-v2", n_envs=1)

eval_callback = EvalCallback(env_val, best_model_save_path="./logs/", log_path="./logs/", eval_freq=500, render=False,  deterministic=True)

# Initialize PPO
model = SAC('MlpPolicy', env, verbose=0)

# Train the model
model.learn(total_timesteps=200000,  progress_bar=True, callback=eval_callback)

# Save the model
model.save("sac_lunar_lander")

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

env.close()
env_val.close()

Output()

Eval num_timesteps=8000, episode_reward=-190.63 +/- 26.01

Episode length: 150.20 +/- 62.44

New best mean reward!

Eval num_timesteps=16000, episode_reward=-231.71 +/- 51.22

...

Eval num_timesteps=192000, episode_reward=-52.05 +/- 27.81

Episode length: 1000.00 +/- 0.00

Eval num_timesteps=200000, episode_reward=-36.50 +/- 26.98

Episode length: 1000.00 +/- 0.00

Mean reward: -45.56 +/- 39.10


In [ ]:
# Load the best model
env = make_vec_env("LunarLanderContinuous-v2", n_envs=1)
best_model_path = "./logs/best_model.zip"
best_model = SAC.load(best_model_path, env=env)

mean_reward, std_reward = evaluate_policy(best_model, env, n_eval_episodes=5)
print(f"Best Model - Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

# Record video of the best model playing CarRacing
env = VecVideoRecorder(env, "./videos/", video_length=2500, record_video_trigger=lambda x: x == 0, name_prefix="best_model_lunar_lander_sac")

obs = env.reset()
for _ in range(2500):
    action, _states = best_model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

env.close()

Best Model - Mean reward: -23.74 +/- 18.73
Saving video to /content/videos/best_model_lunar_lander_sac-step-0-to-step-2500.mp4
Moviepy - Building video /content/videos/best_model_lunar_lander_sac-step-0-to-step-2500.mp4.
Moviepy - Writing video /content/videos/best_model_lunar_lander_sac-step-0-to-step-2500.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/best_model_lunar_lander_sac-step-0-to-step-2500.mp4


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
